In [83]:
import numpy as np
import pandas as pd
import json
import pickle
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import pearsonr, spearmanr
import copy

##### load for plotting (with plotly)

In [2]:
from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
import plotly
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
plotly.io.orca.config.executable = '/Users/chenruduan/opt/anaconda3/envs/mols_newplotly/bin/orca'
init_notebook_mode(connected=True)
glob_layout = go.Layout(
    font=dict(family='Helvetica', size=24, color='black'),
    margin=dict(l=100, r=10, t=10, b=100),
    xaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=1.5, linewidth=1.5, ticklen=10, linecolor='black',
               mirror="allticks", color="black"),
    yaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=1.5, linewidth=1.5, ticklen=10, linecolor='black',
               mirror="allticks", color="black"),
    legend_orientation="v",
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='white',
)
blue = "rgba(0, 0, 255, 1)"
red = "rgba(255, 0, 0, 1)"
green = "rgba(0, 196, 64, 1)"
gray = "rgba(140, 140, 140, 1)"

##### load for ML parts 

In [5]:
from dfa_recommender.net import GatedNetwork, MySoftplus, TiledMultiLayerNN, MLP, finalMLP, ElementalGate
from dfa_recommender.dataset import SubsetDataset
from dfa_recommender.sampler import InfiniteSampler
from dfa_recommender.ml_utils import numpy_to_dataset
import torch
from torch.utils.data import DataLoader

##### DFAs that we considered

In [54]:
base_keys = ["name", "path", "metal",]
functionals = [
    "bp86", "blyp", "pbe",
    "tpss", "scan", "m06-l", "mn15-l",
    "b3p86", "b3pw91", "b3lyp",
    "tpssh", "scan0", "m06", "m06-2x",
    "wb97x", "LRC-wPBEh",
    "b2gpplyp", "pbe0-dh", "dsd-blyp-d3bj", "dsd-pbeb95-d3bj", "dsd-pbep86-d3bj",
]
functionals += ["blyp_hfx_10", "blyp_hfx_20", "blyp_hfx_30", "blyp_hfx_40", "blyp_hfx_50",
                "pbe_hfx_10", "pbe_hfx_20", "pbe_hfx_30", "pbe_hfx_40", "pbe_hfx_50",   
                "scan_hfx_10", "scan_hfx_20", "scan_hfx_30", "scan_hfx_40", "scan_hfx_50", 
                "m06-l_hfx_10", "m06-l_hfx_30", "m06-l_hfx_40", "m06-l_hfx_50", 
                "mn15-l_hfx_10", "mn15-l_hfx_20", "mn15-l_hfx_30", "mn15-l_hfx_40", "mn15-l_hfx_50"]
all_functionals = copy.deepcopy(functionals)

45

### Predict the veritcal spin-splitting energy on the 452 set

In [6]:
torch.set_num_threads(4)
torch.manual_seed(0)
np.random.seed(0)
device = torch.device('cpu')
num_workers = 0

##### set path for relavant data files: a random train/test split, where the model is tested on the in-distribution test data

In [7]:
from pkg_resources import resource_filename, Requirement
basepath = resource_filename(Requirement.parse("dfa_recommender"), "/dfa_recommender/data/")

In [37]:
X_org = pickle.load(open(basepath +  "/X_vss452.pickle", "rb")) ## features 
df_org = pd.read_csv(basepath + "/VSS-452.csv") ## csv file that stores the compuated vert SSE values at different methods
y_scalers = pickle.load(open(basepath +  "/abs-reg-y_scalers.pkl", "rb")) ## sklearn.preprocessing.StandardScaler object created on the stats of training data
tr_inds, te_inds = list(), list()
for ii, val in enumerate(df_org["train"].values):
    if val == 1:
        tr_inds.append(ii)
    elif val == 0:
        te_inds.append(ii)
len(tr_inds), len(te_inds)

/opt/anaconda3/envs/dfa_rec/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



(300, 152)

In [47]:
import os, shutil
if not os.path.isdir("../data/optgeos/VSS-452/"):
    os.makedirs("../data/optgeos/VSS-452/")
for name in df_org["name"].values:
    xyzname = "optgeo_" + name + ".xyz"
    shutil.copy("../data/optgeos/" + xyzname, "../data/optgeos/VSS-452/" + name + ".xyz")

##### predict |DFA - DLPNO-CCSD(T)| vertical spin splitting

In [39]:
res_all = {}
mae_dict = {}
for f in all_functionals:
    bz = 16
    # X_tr, X_te = X_org[super_tr_inds], X_org[super_te_inds]
    # y_t = np.abs(df_org["delta.%s.vertsse"%f].values)
    # y_scaler = y_scalers[f]
    # y_t = y_scalers[f].transform(y_t.reshape(-1, 1)).reshape(-1, )
    # y_tr, y_te = y_t[super_tr_inds], y_t[super_te_inds]
    
    X_tr, X_te = X_org[tr_inds], X_org[te_inds]
    y_t = np.abs(df_org["delta.%s.vertsse"%f].values)
    y_scaler = y_scalers[f]
    y_t = y_scalers[f].transform(y_t.reshape(-1, 1)).reshape(-1, )
    y_tr, y_te = y_t[tr_inds], y_t[te_inds]

    data_tr, data_te = numpy_to_dataset(X_tr, y_tr, regression=True), numpy_to_dataset(X_te, y_te, regression=True)
    tr_l = SubsetDataset(data_tr, list(range(len(data_tr))))
    te_l = SubsetDataset(data_te, list(range(len(data_te))))
    # print("sub labeled dataset length: ", len(tr_l), len(te_l))

    l_tr_iter = iter(DataLoader(tr_l, bz, num_workers=num_workers,
                                sampler=InfiniteSampler(len(tr_l))))
    l_te_iter = iter(DataLoader(te_l, bz, num_workers=num_workers,
                                sampler=InfiniteSampler(len(te_l))))
    te_loader = DataLoader(te_l, len(te_l), num_workers=num_workers)
    tr_l_loader = DataLoader(tr_l, len(tr_l), num_workers=num_workers)
    
    best_model = pickle.load(open(basepath + "/models-trends/mergedG10-abs-reg-%s.pkl"%f, "rb"))
    best_model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for x, y in te_loader:
            _pred = best_model(x.to(device))
            preds.append(_pred.cpu().numpy())
            labels.append(y.cpu().numpy())
    y_t = y_scaler.inverse_transform(labels[0].reshape(-1, 1)).reshape(-1, )
    y_hat = y_scaler.inverse_transform(preds[0].reshape(-1, 1)).reshape(-1, )
    non_nan_inds = np.where(~np.isnan(y_t))[0]
    y_t_super = np.copy(y_t)
    _y_t = y_t[non_nan_inds]
    _y_hat = y_hat[non_nan_inds]
    mae = mean_absolute_error(_y_hat, _y_t)
    scaled_mae = mae/(np.max(_y_t) - np.min(_y_t))
    R2 = r2_score(_y_t, _y_hat)
    rval = pearsonr(_y_t, _y_hat)[0]
    mae_dict[f] = np.float64(mae)
    print(f, "mae: ", round(mae, 5), "scaled mae: ", round(scaled_mae, 5), "R2: ", round(R2, 4), "r val: ", round(rval, 4))
    res_all[f + ".y_t"] = np.abs(y_t)
    res_all[f + ".y_hat"] = np.abs(y_hat)
    res_all["name"] = df_org["name"].values[te_inds]
    res_all["metal"] = df_org["metal"].values[te_inds]
    res_all["dlpno.vertsse"] = df_org["dlpno-CCSD_T.vertsse"].values[te_inds]

bp86 mae:  2.76731 scaled mae:  0.0534 R2:  0.8864 r val:  0.9416
blyp mae:  2.51512 scaled mae:  0.04818 R2:  0.9132 r val:  0.9564
pbe mae:  2.77739 scaled mae:  0.05377 R2:  0.8752 r val:  0.9366
tpss mae:  2.56791 scaled mae:  0.0509 R2:  0.9106 r val:  0.9543
scan mae:  3.13693 scaled mae:  0.05897 R2:  0.8514 r val:  0.926
m06-l mae:  2.63249 scaled mae:  0.08552 R2:  0.7185 r val:  0.8568
mn15-l mae:  2.83456 scaled mae:  0.11524 R2:  0.5165 r val:  0.7307
b3p86 mae:  2.35218 scaled mae:  0.06124 R2:  0.8658 r val:  0.931
b3pw91 mae:  2.37783 scaled mae:  0.06607 R2:  0.8298 r val:  0.9117
b3lyp mae:  2.24489 scaled mae:  0.06127 R2:  0.8655 r val:  0.9312
tpssh mae:  2.64007 scaled mae:  0.05878 R2:  0.8785 r val:  0.9387
scan0 mae:  2.21471 scaled mae:  0.064 R2:  0.7932 r val:  0.8938
m06 mae:  2.44559 scaled mae:  0.11631 R2:  0.5951 r val:  0.7769
m06-2x mae:  1.7109 scaled mae:  0.06533 R2:  0.5049 r val:  0.724
wb97x mae:  2.18326 scaled mae:  0.06723 R2:  0.8384 r val:  

### Analyze

##### sort based on ML predicted |DFA - DLPNO-CCSD(T)| vertical spin splitting to select DFAs

In [61]:
df_res = pd.DataFrame.from_dict(res_all)
len(df_res)

152

In [62]:
removed = []
functionals = list(set(all_functionals).difference(set(removed)))

thresh = 0.0
errs_t, errs_hat, best_fs, lmstds, lower_bound, true_best_fs, ranks = [
], [], [], [], [], [], []
err = {}
for f in functionals:
    err[f] = []
for _, row in df_res.iterrows():
    res_true = [row[f + ".y_t"]
                if not np.isnan(row[f + ".y_t"]) else 1000 for f in functionals]
    fs_true = [x for _, x in sorted(zip(res_true, functionals))]
    res_true = sorted(res_true)
    res_hat = [row[f + ".y_hat"]
               if ((not np.isnan(row[f + ".y_t"]))) else 1000 for f in functionals]
    fs = [x for _, x in sorted(zip(res_hat, functionals))]
    res_hat = sorted(res_hat)
    ranks += [res_true.index(row[fs[0] + ".y_t"])]

    errs_t += [row[fs[0] + ".y_t"]]
    errs_hat += [row[fs[0] + ".y_hat"]]
    best_fs += [fs[0]]
    lower_bound += [row[fs_true[0] + ".y_t"]]
    true_best_fs += [fs_true[0]]

df_sel = pd.DataFrame.from_dict({"errs_t": errs_t, "errs_hat": errs_hat,
                                 "best_fs": best_fs, "lower_bound": lower_bound, "true_best_fs": true_best_fs,
                                 "ranks": ranks, "name": df_res["name"].values,
                                 "dlpno.vertsse": df_res["dlpno.vertsse"].values
                                 })
df_sel = df_sel.sort_values('errs_t')


topx = 6
thresh = 0.0
errs_t, errs_hat, best_fs, lmstds, lower_bound, true_best_fs, ranks = [
], [], [], [], [], [], []
err = {}
for f in functionals:
    err[f] = []
for _, row in df_res.iterrows():
    res_true = [row[f + ".y_t"]
                if not np.isnan(row[f + ".y_t"]) else 1000 for f in functionals]
    fs_true = [x for _, x in sorted(zip(res_true, functionals))]
    res_true = sorted(res_true)
    res_hat = [row[f + ".y_hat"]
               if ((not np.isnan(row[f + ".y_t"]))) else 1000 for f in functionals]
    fs = [x for _, x in sorted(zip(res_hat, functionals))]
    res_hat = sorted(res_hat)
    ranks += [res_true.index(row[fs[0] + ".y_t"])]

    errs_t += [[round(row[fs[ii] + ".y_t"], 2) for ii in range(topx)]]
    errs_hat += [[round(row[fs[ii] + ".y_hat"], 2) for ii in range(topx)]]
    best_fs += [[fs[ii] for ii in range(topx)]]
    lower_bound += [[round(row[fs_true[ii] + ".y_t"], 2)
                     for ii in range(topx)]]
    true_best_fs += [[fs_true[ii] for ii in range(topx)]]
df_sel_top = pd.DataFrame.from_dict({"errs_t": errs_t, "errs_hat": errs_hat, 
                                     "best_fs": best_fs, "lower_bound": lower_bound, "true_best_fs": true_best_fs,
                                     "ranks": ranks, "name": df_res["name"].values,
                                     "dlpno.vertsse": df_res["dlpno.vertsse"].values,
                                     "metal": df_res["metal"].values,
                                     })
df_sel_top = df_sel_top.sort_values('errs_t')


##### absolute error distribution

In [63]:
hist_data = [df_sel['errs_t'].values]
group_labels = [""]
colors = ['black', blue, green, red]
fig = ff.create_distplot(hist_data, group_labels, show_hist=True, colors=colors, bin_size=0.5)
layout = go.Layout()
layout.legend.update(x=.5, y=1, bgcolor="rgba(0,0,0,0)")
layout.update(showlegend=False)
layout.update(width=550, height=500)
layout.update(glob_layout)
layout["xaxis"].update({'title': "abs. err. (kcal/mol)"})
layout["yaxis"].update({'title': "frequency"})
fig.layout.update(layout)
fig.show()

np.mean(df_sel['errs_t'])

2.1270691410234

##### DFA ranks

In [64]:
y = []
for ii in range(48):
    y += [len(df_sel[df_sel["ranks"] == ii])*100./len(df_sel)]
data = [go.Bar(x=list(range(48)),
               y=y, name='all', marker_color='rgba(0, 0, 0, 0.5)', showlegend=False),]
xs=list(range(48))
ys=[np.sum(y[:ii])*100./np.sum(y) for ii in xs]
for ii in range(47):
    data += [go.Scatter(x=[xs[ii], xs[ii]], y=[ys[ii], ys[ii+1]], mode='lines', yaxis="y2", line=dict(color='blue', width=2, dash='solid'), showlegend=False)]
    data += [go.Scatter(x=[xs[ii], xs[ii+1]], y=[ys[ii+1], ys[ii+1]], mode='lines', yaxis="y2", line=dict(color='blue', width=2, dash='solid'), showlegend=False)]
layout = go.Layout()
layout.update(glob_layout)
layout.legend.update(x=1, y=1, bgcolor="rgba(0,0,0,0)")
layout["xaxis"].update({'title': "DFA rank"})
layout["yaxis"].update({'title': "percentage", "mirror": False})
layout.update({"yaxis2": dict(
    title="cumulative percentage",
    titlefont=dict(color="black"),
    tickfont=dict(color="black"),
    anchor="free",
    overlaying="y",
    side="right",
    position=1,
    range=[0, 100],
    showgrid=True,  
    zeroline=True, 
    ticks="inside", 
    showline=True,
    tickwidth=3, 
    linewidth=3, 
    ticklen=10,
#     mirror=True,
)},)
layout.update(width=600, height=500, boxmode='group')
fig = dict(data=data, layout=layout)
iplot(fig)

##### likelihood of top 5 DFAs

In [73]:
### ----top-5----
cutoff = 15
top5_bfs = {}
for f in functionals:
    top5_bfs[f] = 0
for _, row in df_sel_top.iterrows():
    for f in row["best_fs"]:
        top5_bfs[f] += 1
true_top5_bfs = {}
for f in functionals:
    true_top5_bfs[f] = 0
for _, row in df_sel_top.iterrows():
    for f in row["true_best_fs"]:
        true_top5_bfs[f] += 1

for f in top5_bfs:
    top5_bfs[f] += true_top5_bfs[f]*0.01

from collections import Counter
bfs = dict(sorted(top5_bfs.items(), key=lambda item: item[1], reverse=True))
use_bfs_true = {}
for f in list(bfs.keys()):
    use_bfs_true[f] = true_top5_bfs[f]
data = [go.Bar(x=list(bfs.keys())[:cutoff],
               y=np.array(list(bfs.values()))[:cutoff]*600./np.sum(list(bfs.values())), name='rec', marker_color='rgba(0, 0, 255, 0.5)'),
        go.Bar(x=list(use_bfs_true.keys())[:cutoff],
               y=np.array(list(use_bfs_true.values()))[:cutoff]*600./np.sum(list(bfs.values())), name='true', marker_color='rgba(0, 255, 0, 0.5)'),
       ]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "DFA"})
layout["yaxis"].update({'title': "likelihood of top 5", })
layout.update(width=1000, height=500, boxmode='group')
fig = dict(data=data, layout=layout)
iplot(fig)

In [84]:
#---Calculate rank ordering of the recommender selected DFAs and the ground truth---
TL_ranked_DFAs = list(bfs.keys())
bfs_true = dict(sorted(true_top5_bfs.items(), key=lambda item: item[1], reverse=True))
l1 = [ii for ii, _ in enumerate(list(bfs_true.keys()))]
l2 = [list(TL_ranked_DFAs).index(f) for f in list(bfs_true.keys())]
spearmanr(l1, l2)

SpearmanrResult(correlation=0.9469038208168641, pvalue=8.223112687650343e-23)

##### with specific metal

In [94]:
### ----top-5----
cutoff = 15
top5_bfs = {}
_df_sel_top = df_sel_top[df_sel_top["metal"] == "mn"]
for f in functionals:
    top5_bfs[f] = 0
for _, row in _df_sel_top.iterrows():
    for f in row["best_fs"]:
        top5_bfs[f] += 1
true_top5_bfs = {}
for f in functionals:
    true_top5_bfs[f] = 0
for _, row in _df_sel_top.iterrows():
    for f in row["true_best_fs"]:
        true_top5_bfs[f] += 1
        
for f in top5_bfs:
    top5_bfs[f] += true_top5_bfs[f]*0.01

from collections import Counter
bfs = dict(sorted(top5_bfs.items(), key=lambda item: item[1], reverse=True))
use_bfs_true = {}
for f in list(bfs.keys())[:cutoff]:
    use_bfs_true[f] = true_top5_bfs[f]
data = [go.Bar(x=list(bfs.keys())[:cutoff],
               y=np.array(list(bfs.values()))[:cutoff]*600./np.sum(list(bfs.values())), name='rec', marker_color='rgba(0, 0, 255, 0.5)'),
        go.Bar(x=list(use_bfs_true.keys()),
               y=np.array(list(use_bfs_true.values()))*600./np.sum(list(bfs.values())), name='true', marker_color='rgba(0, 255, 0, 0.5)'),
       ]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "DFA"})
layout["yaxis"].update({'title': "likelihood of top 5", })
layout.update(width=1000, height=500, boxmode='group')
fig = dict(data=data, layout=layout)
iplot(fig)
len(_df_sel_top)

38

In [95]:
#---Calculate rank ordering of the recommender selected DFAs and the ground truth---
TL_ranked_DFAs = list(bfs.keys())
bfs_true = dict(sorted(true_top5_bfs.items(), key=lambda item: item[1], reverse=True))
l1 = [ii for ii, _ in enumerate(list(bfs_true.keys()))]
l2 = [list(TL_ranked_DFAs).index(f) for f in list(bfs_true.keys())]
spearmanr(l1, l2)

SpearmanrResult(correlation=0.9382081686429512, pvalue=1.964287320133818e-21)